In [1]:
import redis
import os
import environ
from pathlib import Path
import pickle
import pandas as pd

### Redis の初期設定

In [2]:
BASE_DIR = Path('__file__').resolve().parent
env = environ.Env()
env.read_env(os.path.join(BASE_DIR, ".env"))
HOST_NAME = env("HOST_NAME")
ACCESS_KEY = env("ACCESS_KEY")

In [3]:
r = redis.StrictRedis(host=HOST_NAME,
    port=6380, db=0, password=ACCESS_KEY, ssl=True)

### DataFrameで登録データを作成

In [4]:
df1 = pd.DataFrame(
    [
        ["Do", "Deer"],
        ["Re", "Ray"],
        ["Mi", "Me"],
        ["Fa", "Far"],    
    ],
    columns=["scale", "example"]
)

df1

,scale,example
0,Do,Deer
1,Re,Ray
2,Mi,Me
3,Fa,Far


### Redis に登録

In [5]:
r.flushall() # 全消去

for i in range(len(df1)):
    
    key = df1.iloc[i:i+1]["scale"].values[0]
    value = df1.iloc[i:i+1]["example"].values[0]
    
    r.set(key, value)

In [6]:
for key in r.keys():
    print(key.decode('utf-8'), r.get(key).decode('utf-8'))

Do Deer
Fa Far
Mi Me
Re Ray


### 書き換え

In [7]:
df2 = pd.DataFrame(
    [
        ["Do", "Donuts"],
        ["Fa", "Fight"],    
    ],
    columns=["scale", "example"]
)

df2

,scale,example
0,Do,Donuts
1,Fa,Fight


In [8]:
for i in range(len(df2)):
    
    key = df2.iloc[i:i+1]["scale"].values[0]
    value = df2.iloc[i:i+1]["example"].values[0]
    print(key, value)
    
    r.set(key, value)

Do Donuts
Fa Fight


### 出力

In [9]:
for key in r.keys():
    print(key.decode('utf-8'), r.get(key).decode('utf-8'))

Do Donuts
Fa Fight
Mi Me
Re Ray


### ハッシュ型で登録

In [20]:
pd_dict = dict(zip(df1["scale"], df1['example']))

r.hset("scale", mapping=pd_dict)

0

In [23]:
# すべて出力
print(r.hgetall("scale"))

# Key のみ出力
print(r.hkeys("scale"))

# value のみ出力
print(r.hvals("scale"))

{b'Do': b'Deer', b'Re': b'Ray', b'Mi': b'Me', b'Fa': b'Far'}